In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

In [ ]:
%run othello_game.ipynb
%run othello_ai.ipynb

In [ ]:
import time
import ipywidgets

# Hilfsfunktionen

Der folgende Code beinhaltet Hilfsfunktionen zum Testen und sammeln von Statistiken.

Die Funktion `debug_num_visited_states` berechnet für einen Spielzustand den nächsten Zug mit verschiedenen Algorithmen und misst dabei die benötigte Zeit. Zusätzlich werden mit den globalen Variablen `debug_mm_count` und  `debug_ab_count` die Anzahl der überprüften Zustände gezählt. Diese Zahlen geben einen Überblick darüber, wie viele Zweige durch den Algorithmus ausgeschlossen werden konnten und nicht überprüft werden mussten.

In [ ]:
def debug_num_visited_states(state, depth):
    global transposition_table
    global debug_mm_count
    global debug_ab_count
    global debug_pc_count

    # calculate next move with each algorithm and measure time
    debug_mm_count= 0
    start = time.time()
    ai_make_move(minimax, state, depth, combined_heuristic)
    secs = time.time() - start
    print("Minimax takes", secs, "s and evaluates the heuristic",
          debug_mm_count, "times")
    debug_ab_count= 0
    transposition_table = {}
    start = time.time()
    ai_make_move(alphabeta, state, depth, combined_heuristic)
    secs = time.time() - start
    print("AlphaBeta takes", secs, "s and evaluates the heuristic",
          debug_ab_count, "times")
    debug_ab_count = 0
    transposition_table = {}
    start = time.time()
    ai_make_move_id(alphabeta, state, depth, combined_heuristic)
    secs = time.time() - start
    print("AlphaBeta + ID takes", secs,
          "s and evaluates the heuristic", debug_ab_count, "times")
    debug_pc_count = 0
    transposition_table = {}
    start = time.time()
    ai_make_move(probcut, state, depth, combined_heuristic)
    secs = time.time() - start
    print("ProbCut takes", secs, "s and evaluates the heuristic",
          debug_pc_count, "times")
    
    debug_pc_count = 0
    transposition_table = {}
    start = time.time()
    ai_make_move_id(probcut, state, depth, combined_heuristic)
    secs = time.time() - start
    print("ProbCut + ID takes", secs, "s and evaluates the heuristic",
          debug_pc_count, "times")

Die Funktion `get_statistics` ist dazu da, mehrere Spiele zu berechnen, in denen zwei KIs gegeneinander spielen und Statistiken darüber zu sammeln. `num` ist die Anzahl der Spiele, die durchgeführt werden sollen. Die weiteren Parameter legen fest, welche KIs und welche Heuristiken für die Spieler verwendet werden sollen. Statistiken werden nach jedem Spiel aktualisiert.

Da das Spielfeld nicht gezeichnet werden soll, wird statt `next_move` die Funktion `next_move_blind` verwendet.

In [ ]:
def next_move_blind(state, settings):
    make_move = settings[state.turn]['mode']
    algorithm = settings[state.turn]['algorithm']
    depth = settings[state.turn]['depth']
    heuristic = settings[state.turn]['heuristic']
    state = make_move(algorithm, state, depth, heuristic)
    if not state.game_over:
        next_move_blind(state, settings)

In [ ]:
def play_game(settings):
    state = GameState()
    next_move_blind(state, settings)
    return count_disks(state, BLACK), count_disks(state, WHITE)

In [ ]:
def get_statistics(num, settings):
    status = ipywidgets.widgets.Label()
    display(status)
    result = []
    wins = [0, 0, 0]
    status.value = f'0 / {num} games played, b/d/w: {wins[0]}/{wins[1]}/{wins[2]}'
    try:
        for i in range(num):
            (b, w) = play_game(settings)
            result.append((b, w))
            if b > w:
                wins[0] += 1
            elif w == b:
                wins[1] += 1
            else:
                wins[2] += 1
            status.value = f'{i+1} / {num} games played, b/d/w: {wins[0]}/{wins[1]}/{wins[2]}'
    except KeyboardInterrupt:
        status.value = f'Interrupted: {i} / {num} games played, b/d/w: {wins[0]}/{wins[1]}/{wins[2]}'
    print(result)
    return wins